In [1]:
import pandas as pd
import numpy as np
import sys
import os
os.chdir('/Users/veronica/Documents/GitHub/analysis-tools')
from src import utils, credentials

In [2]:
creds = credentials.creds
engine = utils.create_sql_db(creds)

In [3]:
query = """WITH base as (
SELECT 
  c.contract_id,
  c.store_id,
  c.seller_name,
  c.plan_id,
  c.plan_purchase_price,
  pcrs_contracts.retail_cost_of_plan,
  pcrs_contracts.product_purchase_price,
  c.product_list_price,
  c.plan_sku,
  prod.reimbursement_amount,
  CASE WHEN prod.reimbursement_amount IS NOT NULL THEN prod.reimbursement_amount/100.0 ELSE c.product_list_price END as price_final,
  m.merchant_cut
FROM analytics.contract_facts c
LEFT JOIN analytics.merchant_facts m -- get merchant_cut
ON c.store_id = m.store_id
LEFT JOIN analytics.product_facts prod -- get reimbursement amount
ON c.product_id = prod.product_id AND c.store_id = prod.store_id
AND c.status in ('live', 'expired')
LEFT JOIN analytics_pcrs.pcrs_contracts_sold AS pcrs_contracts
ON c.admin_contract_id = pcrs_contracts.contract_number
)

SELECT
  b.*,
  p.retail_target,
  p.retail_min,
  p.retail_max,
  p.fixed_price,
  pm.cost/100.0 as cost
FROM base b
LEFT JOIN analytics.plans_pricing p -- get retail_target 
USING(plan_id)
LEFT JOIN analytics.plans_matching pm -- get cost (premium) 
ON b.plan_id = pm.plan_id AND b.plan_sku = pm.vendor_sku AND p.price_high = pm.price_high and p.price_low = pm.price_low
WHERE 
b.price_final BETWEEN (p.price_low/100.0) AND (p.price_high/100.0)"""

df = pd.read_sql_query(query, engine)

In [11]:
# Clean Data
# Remove contracts with a fixed price, empty cost value, or where cost is set to 1

In [12]:
df_clean = df[(df['cost'].notnull()) & (df['cost']!= 0.01) & (df['fixed_price'].isnull())]
df_clean

,contract_id,store_id,seller_name,plan_id,plan_purchase_price,retail_cost_of_plan,product_purchase_price,product_list_price,plan_sku,reimbursement_amount,price_final,merchant_cut,retail_target,retail_min,retail_max,fixed_price,cost
0,0c0f00d1-c0c9-4894-b76e-bdac35aeefc1,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-1y,14.99,14.99,160.00,160.00,EXTRPLBG022,16000.0,160.00,0.500,0.09,0.05,0.30,NaN,5.04
1,ab215c19-ce11-4e9f-a77d-3b085c1a40c3,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-1y,14.99,14.99,160.00,160.00,EXTRPLBG022,16000.0,160.00,0.500,0.09,0.05,0.30,NaN,5.04
2,902aa818-6982-49af-af84-17d3b7187b39,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-3y,36.99,36.99,160.00,160.00,EXTRPLBG258,16000.0,160.00,0.500,0.20,0.05,0.30,NaN,14.62
3,abe2da66-7fd4-44d4-8791-a813349446ac,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-2y,24.99,24.99,160.00,160.00,EXTRPLBG140,16000.0,160.00,0.500,0.15,0.05,0.30,NaN,9.78
4,01f18938-1473-4ac2-87ac-d58827934e20,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-1y,14.99,14.99,160.00,160.00,EXTRPLBG022,16000.0,160.00,0.500,0.09,0.05,0.30,NaN,5.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397971,798261b9-b474-45bd-bc1e-aae3d5242316,583c033b-74fd-4620-b64a-ee721355d4ee,L'ange Hair,10001-misc-elec-adh-replace-2y,19.99,19.99,115.00,115.00,EXTRPL72,NaN,115.00,0.400,0.17,0.09,0.51,NaN,8.97
397972,54bfc40a-046f-4577-96c6-75317e67c704,a92122f0-2e8b-47b8-9f29-9efbe3b327bb,Jewlr,A0-AAJEW-3y,29.99,8.49,1.00,1.00,EXTJEW18,NaN,1.00,0.234,0.42,0.05,0.30,NaN,5.55
397973,f703abf4-7214-4517-9ab4-75fc140c36a5,81483a9e-fdfc-4de9-a5a6-239dae52a731,Yale Home,10001-misc-elec-base-replace-1y,29.99,29.99,299.00,299.00,EXTRPL7,NaN,299.00,0.300,0.10,0.05,0.30,NaN,11.56
397975,6456f414-e8bd-43a6-9251-6c5238cc8713,f8920f29-51d9-4c84-930c-ac56c472106b,RealTruck,10001-auto-part-base-replace-2y,5.99,5.99,45.00,45.00,EXTAPART31,NaN,45.00,0.300,0.13,0.07,0.39,NaN,2.35


In [13]:
def calc_target_price(df):
    df['target_price'] = df['retail_target'] * df['product_list_price']
    return df

In [14]:
def calc_min_price(df, extend_cut):
    df[f'min_price_{extend_cut}'] = df['cost'] / (1 - df['merchant_cut'] - extend_cut)
    return df

In [15]:
def calc_diff(df, min_price_col):
    df['price_diff'] = df['target_price'] - df[min_price_col]
    df['price_diff_pct'] = (df['target_price'] - df[min_price_col])/ df[min_price_col]
    return df

In [16]:
def find_price_floor(df, min_price_col):
    df['price_floor'] = np.where(df[min_price_col] > df['target_price'], 1, 0)
    return df

In [17]:
df_price = calc_target_price(df_clean)
df_price = calc_min_price(df_price, 0.10)
df_price = calc_diff(df_price, 'min_price_0.1')
df_price = find_price_floor(df_price, 'min_price_0.1')
df_price.head()

/Users/veronica/env/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/veronica/env/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,contract_id,store_id,seller_name,plan_id,plan_purchase_price,retail_cost_of_plan,product_purchase_price,product_list_price,plan_sku,reimbursement_amount,...,retail_target,retail_min,retail_max,fixed_price,cost,target_price,min_price_0.1,price_diff,price_diff_pct,price_floor
0,0c0f00d1-c0c9-4894-b76e-bdac35aeefc1,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-1y,14.99,14.99,160.0,160.0,EXTRPLBG022,16000.0,...,0.09,0.05,0.3,NaN,5.04,14.4,12.60,1.80,0.142857,0
1,ab215c19-ce11-4e9f-a77d-3b085c1a40c3,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-1y,14.99,14.99,160.0,160.0,EXTRPLBG022,16000.0,...,0.09,0.05,0.3,NaN,5.04,14.4,12.60,1.80,0.142857,0
2,902aa818-6982-49af-af84-17d3b7187b39,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-3y,36.99,36.99,160.0,160.0,EXTRPLBG258,16000.0,...,0.20,0.05,0.3,NaN,14.62,32.0,36.55,-4.55,-0.124487,1
3,abe2da66-7fd4-44d4-8791-a813349446ac,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-2y,24.99,24.99,160.0,160.0,EXTRPLBG140,16000.0,...,0.15,0.05,0.3,NaN,9.78,24.0,24.45,-0.45,-0.018405,1
4,01f18938-1473-4ac2-87ac-d58827934e20,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-1y,14.99,14.99,160.0,160.0,EXTRPLBG022,16000.0,...,0.09,0.05,0.3,NaN,5.04,14.4,12.60,1.80,0.142857,0


In [18]:
len(df_price)

213939

In [34]:
df_price['target_price_high'] = df_price['product_list_price'] * df_price['retail_max']
df_price['target_price_low'] = df_price['product_list_price'] * df_price['retail_min']

/Users/veronica/env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/veronica/env/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [35]:
df_pf = df_price[df_price['price_floor']==1]

In [36]:
len(df_pf)

81036

In [37]:
len(df_pf['store_id'].unique())

169

In [38]:
df_pf['seller_name'].value_counts().head(20)

BlendJet                  16492
JBL-US                    11680
Motosport                  8807
RealTruck                  6775
L'ange Hair                5278
Carandtruckremotes.com     3888
Skullcandy                 2558
Jewlr                      2381
EightVape                  1900
DailySale                  1649
Parts Dr                   1340
RaceDayQuads               1253
Jegs High Performance      1167
Jetson Electric Bikes      1133
Logitech                   1030
UE Custom                   716
PCLiquidations              708
Wyze Ecommerce              696
Blackstone Products         657
Backcountry.com             616
Name: seller_name, dtype: int64

In [39]:
df_pf['retail_min_diff'] = abs(df_pf['retail_cost_of_plan'] - df_pf['min_price_0.1'])
df_pf['min_price_int'] = round(df_pf['min_price_0.1'],0)
df_pf['retail_min_int_diff'] = abs(df_pf['retail_cost_of_plan'] - df_pf['min_price_int'])
df_pf['retail_target_diff'] = abs(df_pf['retail_cost_of_plan'] - df_pf['target_price'])
df_pf['min_target_diff'] = abs(df_pf['min_price_0.1'] - df_pf['target_price'])

/Users/veronica/env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/veronica/env/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/veronica/env/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

In [40]:
df_pf[df_pf['product_purchase_price']==1].head()

,contract_id,store_id,seller_name,plan_id,plan_purchase_price,retail_cost_of_plan,product_purchase_price,product_list_price,plan_sku,reimbursement_amount,...,price_diff,price_diff_pct,price_floor,target_price_high,target_price_low,retail_min_diff,min_price_int,retail_min_int_diff,retail_target_diff,min_target_diff
45782,9cdb59bf-5bb9-4c53-9dcc-1be6839926e1,4939fa74-dfef-44ef-a548-67545b390ff8,Lee Fiori,A0-AAJEW-5y,54.99,12.99,1.0,1.0,EXTJEW35,100.0,...,-12.282913,-0.951212,1,0.3,0.05,0.077087,13.0,0.01,12.36,12.282913
45783,e299eb18-60e0-412a-9bca-75674d64b7aa,a92122f0-2e8b-47b8-9f29-9efbe3b327bb,Jewlr,A0-AAJEW-5y,31.99,12.99,1.0,1.0,EXTJEW35,100.0,...,-12.282913,-0.951212,1,0.3,0.05,0.077087,13.0,0.01,12.36,12.282913
45784,0370341c-c19e-46b8-98c7-eebf9c30eb74,a92122f0-2e8b-47b8-9f29-9efbe3b327bb,Jewlr,A1-AAJEW-2y,20.99,5.49,1.0,1.0,EXTJEW1,100.0,...,-4.950210,-0.950098,1,0.3,0.05,0.279790,5.0,0.49,5.23,4.950210
45785,3901bd51-e5ba-4d9e-b902-58878b2d5c42,a92122f0-2e8b-47b8-9f29-9efbe3b327bb,Jewlr,A0-AAJEW-2y,14.99,14.99,1.0,1.0,EXTJEW1,100.0,...,-5.040210,-0.967372,1,0.3,0.05,9.779790,5.0,9.99,14.82,5.040210
45786,72b32cf4-8479-4dd7-8aa8-9876937c8fba,a92122f0-2e8b-47b8-9f29-9efbe3b327bb,Jewlr,A1-AAJEW-2y,21.99,5.49,1.0,1.0,EXTJEW1,100.0,...,-4.950210,-0.950098,1,0.3,0.05,0.279790,5.0,0.49,5.23,4.950210


In [41]:
df_pf_clean = df_pf[df_pf['product_purchase_price']!=1]
df_pf_clean

,contract_id,store_id,seller_name,plan_id,plan_purchase_price,retail_cost_of_plan,product_purchase_price,product_list_price,plan_sku,reimbursement_amount,...,price_diff,price_diff_pct,price_floor,target_price_high,target_price_low,retail_min_diff,min_price_int,retail_min_int_diff,retail_target_diff,min_target_diff
2,902aa818-6982-49af-af84-17d3b7187b39,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-3y,36.99,36.99,160.00,160.00,EXTRPLBG258,16000.0,...,-4.5500,-0.124487,1,48.0000,8.0000,0.440,37.0,0.01,4.9900,4.5500
3,abe2da66-7fd4-44d4-8791-a813349446ac,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-2y,24.99,24.99,160.00,160.00,EXTRPLBG140,16000.0,...,-0.4500,-0.018405,1,48.0000,8.0000,0.540,24.0,0.99,0.9900,0.4500
7,796b685a-7a0e-4054-bc1b-abe8f7a4c17a,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-2y,24.99,24.99,160.00,160.00,EXTRPLBG140,16000.0,...,-0.4500,-0.018405,1,48.0000,8.0000,0.540,24.0,0.99,0.9900,0.4500
9,966564c4-ff7e-4fcd-95d2-f1387afcbf66,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-2y,22.99,22.99,149.99,149.99,EXTRPLBG138,14999.0,...,-0.4515,-0.019673,1,44.9970,7.4995,0.040,23.0,0.01,0.4915,0.4515
12,621813a1-b836-45d6-834f-940ed5bc9705,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-2y,22.99,22.99,149.99,149.99,EXTRPLBG138,14999.0,...,-0.4515,-0.019673,1,44.9970,7.4995,0.040,23.0,0.01,0.4915,0.4515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397961,193477b6-b98d-4674-8803-7a0c486a0d74,194b316d-c59c-4db1-b19e-347ebe7b182b,"VORO MOTORS, Inc.",10001-electric-mobility-base-replace-3y,999.00,999.00,2999.00,2999.00,EXTELS78,NaN,...,-411.7300,-0.432694,1,2639.1200,479.8400,47.450,952.0,47.00,459.1800,411.7300
397964,9682a9d3-2979-4656-a540-12c1806995ba,750e84cb-cdc5-4142-94da-e0115f88f162,JBL-US,10001-portable-audio-base-replace-1y,11.99,11.99,99.95,99.95,EXTPORAUD2,NaN,...,-1.3300,-0.117439,1,29.9850,4.9975,0.665,11.0,0.99,1.9950,1.3300
397967,f4057be7-f105-46f1-8cda-5bb868e35385,750e84cb-cdc5-4142-94da-e0115f88f162,JBL-US,10001-portable-audio-base-replace-2y,22.99,22.99,99.95,99.95,EXTPORAUD7,NaN,...,-4.0090,-0.182227,1,53.9730,8.9955,0.990,22.0,0.99,4.9990,4.0090
397970,b120e5a1-3c7a-4a0d-96cf-2ef2009abbb8,750e84cb-cdc5-4142-94da-e0115f88f162,JBL-US,10001-portable-audio-base-replace-1y,15.99,15.99,119.95,119.95,EXTPORAUD3,NaN,...,-3.7300,-0.237202,1,35.9850,5.9975,0.265,16.0,0.01,3.9950,3.7300


In [42]:
df_pf_clean['seller_name'].value_counts().head(10)

BlendJet                  16492
JBL-US                    11680
Motosport                  8807
RealTruck                  6775
L'ange Hair                5278
Carandtruckremotes.com     3888
Skullcandy                 2558
EightVape                  1900
DailySale                  1649
Parts Dr                   1340
Name: seller_name, dtype: int64

In [43]:
col_list = ['plan_purchase_price', 'retail_cost_of_plan', 'target_price', 'min_price_0.1', 'min_price_int', 'product_list_price', 'price_final', 'merchant_cut', 'cost']

In [45]:
diff_anom = df_pf_clean[(df_pf_clean['retail_target_diff'] < df_pf_clean['retail_min_diff'])].sort_values(by='retail_target_diff', ascending=False)

diff_anom[col_list+['target_price_high', 'target_price_low']]

,plan_purchase_price,retail_cost_of_plan,target_price,min_price_0.1,min_price_int,product_list_price,price_final,merchant_cut,cost,target_price_high,target_price_low
250738,179.00,179.00,108.6300,272.857143,273.0,1086.30,1086.30,0.55,95.50,325.8900,54.3150
203316,149.99,149.99,207.9974,225.320000,225.0,799.99,799.99,0.40,112.66,615.9923,103.9987
357818,549.00,549.00,503.9991,600.866667,601.0,5599.99,5599.99,0.60,180.26,3023.9946,503.9991
209657,269.00,269.00,311.7400,326.933333,327.0,1199.00,1199.00,0.30,196.16,923.2300,155.8700
124486,649.00,649.00,609.6974,883.366667,883.0,2344.99,2344.99,0.60,265.01,1782.1924,304.8487
...,...,...,...,...,...,...,...,...,...,...,...
295566,29.99,29.99,29.9900,33.875000,34.0,149.95,149.95,0.50,13.55,44.9850,7.4975
51990,29.99,29.99,29.9900,33.875000,34.0,149.95,149.95,0.50,13.55,44.9850,7.4975
51991,29.99,29.99,29.9900,33.875000,34.0,149.95,149.95,0.50,13.55,44.9850,7.4975
51995,29.99,29.99,29.9900,33.875000,34.0,149.95,149.95,0.50,13.55,44.9850,7.4975


In [29]:
diff_anom['seller_name'].value_counts().head(20)

BlendJet                  16491
Motosport                  4152
DailySale                  1011
JBL-US                      555
AutoAnything                490
Backcountry.com             145
Shop Indoor Golf            112
Competitive Cyclist          82
MoissaniteCo                 77
Jegs High Performance        64
Zebit                        56
August Home                  44
RealTruck                    24
NewAir                       18
PADO                         16
Smoke Cartel                 15
White Duck Outdoors          11
Evolve Skateboards USA        9
Waterbox Aquariums            9
Steep and Cheap               6
Name: seller_name, dtype: int64

In [30]:
df_pf_final = df_pf_clean[df_pf_clean['retail_target_diff'] > df_pf_clean['retail_min_diff']]
df_pf_final[col_list]

,plan_purchase_price,retail_cost_of_plan,target_price,min_price_0.1,min_price_int,product_list_price,price_final,merchant_cut,cost
2,36.99,36.99,32.0000,36.550,37.0,160.00,160.00,0.5,14.62
3,24.99,24.99,24.0000,24.450,24.0,160.00,160.00,0.5,9.78
7,24.99,24.99,24.0000,24.450,24.0,160.00,160.00,0.5,9.78
9,22.99,22.99,22.4985,22.950,23.0,149.99,149.99,0.5,9.18
12,22.99,22.99,22.4985,22.950,23.0,149.99,149.99,0.5,9.18
...,...,...,...,...,...,...,...,...,...
397961,999.00,999.00,539.8200,951.550,952.0,2999.00,2999.00,0.3,570.93
397964,11.99,11.99,9.9950,11.325,11.0,99.95,99.95,0.5,4.53
397967,22.99,22.99,17.9910,22.000,22.0,99.95,99.95,0.5,8.80
397970,15.99,15.99,11.9950,15.725,16.0,119.95,119.95,0.5,6.29


In [46]:
col         = 'retail_cost_of_plan'
conditions  = [df_pf_clean[col] < 5, (df_pf_clean[col] < 10) & (df_pf_clean[col] >= 5),
              (df_pf_clean[col] < 50) & (df_pf_clean[col] >= 10),
              (df_pf_clean[col] < 100) & (df_pf_clean[col] >= 50),
              (df_pf_clean[col] < 500) & (df_pf_clean[col] >= 100),
              (df_pf_clean[col] < 1000) & (df_pf_clean[col] >= 500),
              (df_pf_clean[col] < 5000) & (df_pf_clean[col] >= 1000),
              (df_pf_clean[col] < 10000) & (df_pf_clean[col] >= 5000),
              (df_pf_clean[col] < 50000) & (df_pf_clean[col] >= 10000)]
choices     = [0.10, 0.50, 1.0, 5.0, 10.0, 50.0, 100, 500, 1000]
df_pf_clean['diff_thresh'] = np.select(conditions, choices, default=np.nan)

/Users/veronica/env/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [47]:
df_pf_clean['diff_flag'] = np.where(df_pf_clean['retail_min_int_diff'] > df_pf_clean['diff_thresh'], 1, 0)
df_pf_clean

/Users/veronica/env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,contract_id,store_id,seller_name,plan_id,plan_purchase_price,retail_cost_of_plan,product_purchase_price,product_list_price,plan_sku,reimbursement_amount,...,price_floor,target_price_high,target_price_low,retail_min_diff,min_price_int,retail_min_int_diff,retail_target_diff,min_target_diff,diff_thresh,diff_flag
2,902aa818-6982-49af-af84-17d3b7187b39,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-3y,36.99,36.99,160.00,160.00,EXTRPLBG258,16000.0,...,1,48.0000,8.0000,0.440,37.0,0.01,4.9900,4.5500,1.0,0
3,abe2da66-7fd4-44d4-8791-a813349446ac,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-2y,24.99,24.99,160.00,160.00,EXTRPLBG140,16000.0,...,1,48.0000,8.0000,0.540,24.0,0.99,0.9900,0.4500,1.0,0
7,796b685a-7a0e-4054-bc1b-abe8f7a4c17a,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-2y,24.99,24.99,160.00,160.00,EXTRPLBG140,16000.0,...,1,48.0000,8.0000,0.540,24.0,0.99,0.9900,0.4500,1.0,0
9,966564c4-ff7e-4fcd-95d2-f1387afcbf66,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-2y,22.99,22.99,149.99,149.99,EXTRPLBG138,14999.0,...,1,44.9970,7.4995,0.040,23.0,0.01,0.4915,0.4515,1.0,0
12,621813a1-b836-45d6-834f-940ed5bc9705,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-2y,22.99,22.99,149.99,149.99,EXTRPLBG138,14999.0,...,1,44.9970,7.4995,0.040,23.0,0.01,0.4915,0.4515,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397961,193477b6-b98d-4674-8803-7a0c486a0d74,194b316d-c59c-4db1-b19e-347ebe7b182b,"VORO MOTORS, Inc.",10001-electric-mobility-base-replace-3y,999.00,999.00,2999.00,2999.00,EXTELS78,NaN,...,1,2639.1200,479.8400,47.450,952.0,47.00,459.1800,411.7300,50.0,0
397964,9682a9d3-2979-4656-a540-12c1806995ba,750e84cb-cdc5-4142-94da-e0115f88f162,JBL-US,10001-portable-audio-base-replace-1y,11.99,11.99,99.95,99.95,EXTPORAUD2,NaN,...,1,29.9850,4.9975,0.665,11.0,0.99,1.9950,1.3300,1.0,0
397967,f4057be7-f105-46f1-8cda-5bb868e35385,750e84cb-cdc5-4142-94da-e0115f88f162,JBL-US,10001-portable-audio-base-replace-2y,22.99,22.99,99.95,99.95,EXTPORAUD7,NaN,...,1,53.9730,8.9955,0.990,22.0,0.99,4.9990,4.0090,1.0,0
397970,b120e5a1-3c7a-4a0d-96cf-2ef2009abbb8,750e84cb-cdc5-4142-94da-e0115f88f162,JBL-US,10001-portable-audio-base-replace-1y,15.99,15.99,119.95,119.95,EXTPORAUD3,NaN,...,1,35.9850,5.9975,0.265,16.0,0.01,3.9950,3.7300,1.0,0


In [49]:
df_check = df_pf_clean[df_pf_clean['diff_flag']==1].sort_values(by='retail_min_diff', ascending=False)

df_check[col_list+['diff_thresh','retail_min_diff','target_price_high', 'target_price_low']]

,plan_purchase_price,retail_cost_of_plan,target_price,min_price_0.1,min_price_int,product_list_price,price_final,merchant_cut,cost,diff_thresh,retail_min_diff,target_price_high,target_price_low
357459,499.00,4990.00,166.5000,291.360000,291.0,1850.00,1850.00,0.65,72.84,100.0,4698.640000,555.0000,92.5000
124486,649.00,649.00,609.6974,883.366667,883.0,2344.99,2344.99,0.60,265.01,50.0,234.366667,1782.1924,304.8487
378629,649.00,649.00,609.6974,883.366667,883.0,2344.99,2344.99,0.60,265.01,50.0,234.366667,1782.1924,304.8487
237048,949.00,949.00,948.9987,1120.200000,1120.0,7299.99,7599.99,0.60,336.06,50.0,171.200000,5620.9923,948.9987
14269,549.00,549.00,518.7000,688.433333,688.0,1995.00,1995.00,0.60,206.53,50.0,139.433333,1516.2000,259.3500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
288706,3.59,3.59,2.9950,3.583333,4.0,29.95,29.95,0.30,2.15,0.1,0.006667,8.9850,1.4975
288711,3.59,3.59,2.9950,3.583333,4.0,29.95,29.95,0.30,2.15,0.1,0.006667,8.9850,1.4975
289301,3.59,3.59,3.4990,3.583333,4.0,34.99,34.99,0.30,2.15,0.1,0.006667,10.4970,1.7495
147757,3.59,3.59,2.9950,3.583333,4.0,29.95,29.95,0.30,2.15,0.1,0.006667,8.9850,1.4975
